In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#	Select Important Features and create a dataframe 

In [17]:
#droped the 'ever_married' and 'Residence_type' column and loaded the csv on dataFrame
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
data=data[['gender',	'age',	'hypertension'	,'heart_disease',	'work_type'	,'avg_glucose_level','bmi','smoking_status','stroke']]
data.head()

,gender,age,hypertension,heart_disease,work_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Private,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Self-employed,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Private,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Private,171.23,34.4,smokes,1
4,Female,79.0,1,0,Self-employed,174.12,24.0,never smoked,1


In [18]:
#fiels the NaN fields of bmi with Median of entire bmi column
data['bmi']=data['bmi'].fillna(data['bmi'].median())


In [19]:
#Removed outliers from age and bmi columns
cols=['age','bmi']
data['zscore']=(data.age-data.age.mean()  )   / data.age.std()
data=data[~(data['zscore']<-1.60)]
data.drop('zscore',axis=1,inplace=True)


In [20]:
#MinMaxScalar preprocessing done on the columns to bring the values of the columns in certain range
from sklearn.preprocessing import MinMaxScaler
scalar=MinMaxScaler()
data[['avg_glucose_level','bmi']]=scalar.fit_transform(data[['avg_glucose_level','bmi']])

In [21]:
#LabelEncoder applied on genders  bexause it contains only 3 distint values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['gender'] = le.fit_transform(data['gender'])


In [22]:
import numpy as np
#applying onehot encoding becaues both the columns had more than 2 distinct values
x=pd.get_dummies(data,columns=['smoking_status','work_type'])
data=pd.concat([data,x],axis=1)
data=data.drop(['smoking_status','work_type'],axis=1)
data = data.loc[:,~data.columns.duplicated()].copy()


In [23]:
l=data.pop('stroke')
data['stroke']=l


In [24]:
from sklearn.feature_selection import SelectKBest,f_classif
selector = SelectKBest(score_func=f_classif)
fits = selector.fit(data.drop("stroke",axis=1),data["stroke"])
x=pd.DataFrame(fits.scores_)
columns = pd.DataFrame(data.drop('stroke',axis=1).columns)
fscores = pd.concat([columns,x],axis=1)
fscores.columns = ['Attribute','Score']
fscores.sort_values(by='Score',ascending=False)

,Attribute,Score
1,age,313.893987
3,heart_disease,84.048571
4,avg_glucose_level,81.711223
2,hypertension,73.089170
14,work_type_children,16.324191
7,smoking_status_formerly smoked,15.687008
13,work_type_Self-employed,14.641421
6,smoking_status_Unknown,5.285087
8,smoking_status_never smoked,1.473690
11,work_type_Never_worked,1.221706


In [25]:
data['bmi']=data['bmi'].fillna(data['bmi'].median())
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values


In [26]:
#SMOTE is a machine learning technique that solves problems that occur when using an imbalanced data set
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
x, y = oversample.fit_resample(x, y)

##	Split Data to train, test, validate 

In [27]:
#Spliting the dataset in test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)


In [28]:
print(len(X_test))
print(len(X_train))

2243
6729


In [29]:
len(X_test)

2243

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
cross_val_score(RandomForestClassifier(n_estimators=10),X_train,y_train)

array([0.94502229, 0.95319465, 0.95913819, 0.94799406, 0.95910781])

In [31]:
cross_val_score(SVC(),X_train,y_train)

array([0.74517088, 0.76374443, 0.75780089, 0.7384844 , 0.75167286])

In [32]:
cross_val_score(KNeighborsClassifier(),X_train,y_train)

array([0.86998514, 0.89524517, 0.90044577, 0.87295691, 0.88550186])

In [33]:
cross_val_score(DecisionTreeClassifier(),X_train,y_train)

array([0.91307578, 0.93684993, 0.92124814, 0.9115899 , 0.92267658])

In [34]:
#Declared the features for tunning og the random forest algo

n_estimatiors=[int(x) for x in np.linspace(start=150,stop=200,num=5)]
max_features=[ 'sqrt', 'log2']
max_depth=[int(x) for x in np.linspace(5, 10, num = 2)] 
min_sample_split=[5,9]
min_sample_leaf=[1,2]
bootstrap=[True,False]

param_grid={
    'n_estimators':n_estimatiors,
    'max_features':max_features,
    'max_depth':max_depth,
    'min_samples_split':min_sample_split,
    'min_samples_leaf':min_sample_leaf,
    'bootstrap':bootstrap
    
}


## Train Model 

In [35]:
#Trained the dataset on Random Forest and used GridSearchCV for aTunning
#IT tested the algorithm on 3*160=480 different parameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
classifier = RandomForestClassifier(class_weight='balanced')
classifier=GridSearchCV(estimator=classifier,param_grid=param_grid,cv=3,verbose=2,n_jobs=4 )
classifier.fit(X_train, y_train)

Fitting 3 folds for each of 160 candidates, totalling 480 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(class_weight='balanced'),
             n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [5, 10],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [5, 9],
                         'n_estimators': [150, 162, 175, 187, 200]},
             verbose=2)

In [36]:
classifier.best_estimator_

RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=10,
                       max_features='log2', min_samples_split=9,
                       n_estimators=150)

In [37]:
print (f'Train Accuracy - : {classifier.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {classifier.score(X_test,y_test):.3f}')

Train Accuracy - : 0.956
Test Accuracy - : 0.930


In [38]:
y_pred = classifier.predict(X_test)
print(y_pred)

[0 0 0 ... 1 1 1]


In [39]:
#Finded the accuracy on the test cases
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
print("confusion matrix->   ") 
print(confusion_matrix(y_test, y_pred))

print("F1score ",f1_score(y_test,y_pred))
print("Accuracy",accuracy_score(y_test, y_pred))

confusion matrix->   
[[1031   93]
 [  63 1056]]
F1score  0.9312169312169313
Accuracy 0.9304502897904592


In [40]:
#Created the pickle file
import pickle
with open('model_pkl', 'wb') as files:
    pickle.dump(classifier, files)

In [41]:
with open('model_pkl' , 'rb') as f:
    lr = pickle.load(f)
y_val_predict=lr.predict(X_test)



In [42]:
#created the pandas dataframe for the csv creation
rd=pd.DataFrame(X_test,columns=['gender', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level',
       'bmi', 'smoking_status_Unknown',
       'smoking_status_formerly smoked', 'smoking_status_never smoked',
       'smoking_status_smokes', 'work_type_0', 'work_type_1', 'work_type_2',
       'work_type_3', 'work_type_4'])
rd['Stroke']=y_test
rd['stroke_predicted']=y_pred

In [43]:
rd.to_csv('validation-predicted.csv', index=False)